In [1]:
!python --version

Python 3.12.9


# Imports

In [2]:
import pandas as pd
import numpy as np
import os
import json
import re
import time
from collections import Counter
import warnings
warnings.filterwarnings('ignore')

# AWS
import boto3
from botocore import UNSIGNED
from botocore.client import Config

import sagemaker
from pyathena import connect
from pyathena.pandas.cursor import PandasCursor

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/sagemaker-user/.config/sagemaker/config.yaml


# Connect to S3

In [3]:
try:
    account_id = boto3.client("sts").get_caller_identity()["Account"]
    if account_id is not "":
        print("✅ Successfuly retrieved account information.")
except Exception as e:
    print(f"❌ Cannot connect to s3 client: {e}")

✅ Successfuly retrieved account information.


In [4]:
REGION = "us-east-1"

# Public dataset bucket
DATA_BUCKET = "yelp-aai540-group6"
BASE_PREFIX = "yelp-dataset/json/"

ATHENA_BUCKET = DATA_BUCKET + f"-athena-{account_id}"
parquet_prefix = "parquet/"

# Athena MUST have a writable results location (a bucket you own)
ATHENA_RESULTS_S3 = f"s3://{ATHENA_BUCKET}/athena-results/"

ATHENA_DB = "yelp"

In [5]:
s3_public = boto3.client("s3", region_name=REGION, config=Config(signature_version=UNSIGNED))
s3_private = boto3.client("s3", region_name=REGION)
athena = boto3.client("athena", region_name=REGION)

### Save bucket information to state

In [6]:
%store -r REGION
%store -r DATA_BUCKET
%store -r ATHENA_BUCKET
%store -r ATHENA_RESULTS_S3
%store -r ATHENA_DB

print("S3 region:", REGION)
print("S3 raw JSON bucket:", DATA_BUCKET)
print("S3 Athena bucket:", ATHENA_BUCKET)
print("S3 Athena results:", ATHENA_RESULTS_S3)
print("S3 Athena DB:", ATHENA_DB)

no stored variable or alias REGION
no stored variable or alias DATA_BUCKET
no stored variable or alias ATHENA_BUCKET
no stored variable or alias ATHENA_RESULTS_S3
no stored variable or alias ATHENA_DB
S3 region: us-east-1
S3 raw JSON bucket: yelp-aai540-group6
S3 Athena bucket: yelp-aai540-group6-athena-297628177412
S3 Athena results: s3://yelp-aai540-group6-athena-297628177412/athena-results/
S3 Athena DB: yelp


## Define filenames

These are the JSON file names as they appear in the public s3 bucket.

In [7]:
FILES = {
    "business": "yelp_academic_dataset_business.json",
    "review": "yelp_academic_dataset_review.json",
    "user": "yelp_academic_dataset_user.json",
    "checkin": "yelp_academic_dataset_checkin.json",
    "tip": "yelp_academic_dataset_tip.json",
}

OBJECT_KEYS = {t: f"{BASE_PREFIX}{fname}" for t, fname in FILES.items()}

for t, k in OBJECT_KEYS.items():
    print(f"{t:8} -> s3://{DATA_BUCKET}.s3.{REGION}.amazonaws.com/{k}")

# destination prefixes (directories)
DEST_PREFIXES = {
    table: f"{BASE_PREFIX}{table}/"
    for table in FILES
}
print(DEST_PREFIXES)

business -> s3://yelp-aai540-group6.s3.us-east-1.amazonaws.com/yelp-dataset/json/yelp_academic_dataset_business.json
review   -> s3://yelp-aai540-group6.s3.us-east-1.amazonaws.com/yelp-dataset/json/yelp_academic_dataset_review.json
user     -> s3://yelp-aai540-group6.s3.us-east-1.amazonaws.com/yelp-dataset/json/yelp_academic_dataset_user.json
checkin  -> s3://yelp-aai540-group6.s3.us-east-1.amazonaws.com/yelp-dataset/json/yelp_academic_dataset_checkin.json
tip      -> s3://yelp-aai540-group6.s3.us-east-1.amazonaws.com/yelp-dataset/json/yelp_academic_dataset_tip.json
{'business': 'yelp-dataset/json/business/', 'review': 'yelp-dataset/json/review/', 'user': 'yelp-dataset/json/user/', 'checkin': 'yelp-dataset/json/checkin/', 'tip': 'yelp-dataset/json/tip/'}


## Verify Connectivity to the bucket for each file

In [8]:
def verify_public_object(bucket: str, key: str):
    try:
        s3_public.head_object(Bucket=bucket, Key=key)
        return True
    except Exception as e:
        print(f"❌ Cannot access s3://{bucket}/{key}\n   Error: {e}")
        return False

all_ok = True
for t, key in OBJECT_KEYS.items():
    ok = verify_public_object(DATA_BUCKET, key)
    print(("✅" if ok else "❌"), t, key)
    all_ok = all_ok and ok

if not all_ok:
    raise RuntimeError("One or more files were not accessible. Check names/paths.")

✅ business yelp-dataset/json/yelp_academic_dataset_business.json
✅ review yelp-dataset/json/yelp_academic_dataset_review.json
✅ user yelp-dataset/json/yelp_academic_dataset_user.json
✅ checkin yelp-dataset/json/yelp_academic_dataset_checkin.json
✅ tip yelp-dataset/json/yelp_academic_dataset_tip.json


## Helper function to run a SQL query in Athena

This takes a query and a database and returns the athena response as a pandas dataframe.

In [9]:
def run_athena_query(query, database):
    params = {
        "QueryString": query,
        "ResultConfiguration": {"OutputLocation": ATHENA_RESULTS_S3},
    }
    if database:
        params["QueryExecutionContext"] = {"Database": database}

    # Start query
    qx = athena.start_query_execution(**params)
    qid = qx["QueryExecutionId"]

    # Wait for completion
    while True:
        resp = athena.get_query_execution(QueryExecutionId=qid)
        state = resp["QueryExecution"]["Status"]["State"]
        if state in ("SUCCEEDED", "FAILED", "CANCELLED"):
            break
        time.sleep(1)

    if state != "SUCCEEDED":
        reason = resp["QueryExecution"]["Status"].get("StateChangeReason", "")
        raise RuntimeError(f"Athena query {state}: {reason}\n\nQuery:\n{query}")

    # Fetch results
    paginator = athena.get_paginator("get_query_results")
    rows = []

    for page in paginator.paginate(QueryExecutionId=qid):
        for row in page["ResultSet"]["Rows"]:
            rows.append([c.get("VarCharValue") for c in row["Data"]])

    # If no rows (DDL statements)
    if not rows:
        return pd.DataFrame()

    # First row is header
    header = rows[0]
    data = rows[1:]

    return pd.DataFrame(data, columns=header)


# Create the Yelp Athena database 

In [10]:
run_athena_query(f"CREATE DATABASE IF NOT EXISTS {ATHENA_DB};", ATHENA_DB)
print(f"✅ Database ready: {ATHENA_DB}")

✅ Database ready: yelp


## Setup a staging area for the data

This cell **copies (streams) public Yelp JSON files from a read-only S3 bucket into a writable S3 bucket** that we control, organizing them into clean, table-specific prefixes that Athena can query reliably.


### Why this is necessary
- The source bucket is **public-read only**, so we cannot write Athena outputs or derived tables there.
- Athena requires:
  - a **directory (prefix)**, not a single file, for table `LOCATION`s
  - a **writable bucket** for query execution and CTAS output
- Staging the data gives us full control without modifying the original dataset.

### Result
After this cell runs:
- All Yelp JSON files exist in a **writable S3 bucket**
- Each dataset lives in its own directory (Athena-compatible)
- We can safely create external tables, views, and Parquet CTAS outputs

The original public dataset remains unchanged.

In [11]:
import boto3
from botocore import UNSIGNED
from botocore.client import Config


BASE_RESULTS_PREFIX = "athena-results/"

STAGING_ROOT = BASE_RESULTS_PREFIX.rstrip("/") + "-staging/yelp/json/"

dest_locations = {}

for table, filename in FILES.items():
    src_key = f"{BASE_PREFIX}{filename}"
    dst_key = f"{STAGING_ROOT}{table}/{filename}"

    print(f"Staging {table}: s3://{DATA_BUCKET}/{src_key} -> s3://{ATHENA_BUCKET}/{dst_key}")

    obj = s3_public.get_object(Bucket=DATA_BUCKET, Key=src_key)
    s3_private.upload_fileobj(obj["Body"], ATHENA_BUCKET, dst_key)

    dest_locations[table] = f"s3://{ATHENA_BUCKET}/{STAGING_ROOT}{table}/"

print("\n✅ Done staging. Use these Athena LOCATION prefixes:")
for t, loc in dest_locations.items():
    print(f"{t:8} -> {loc}")


Staging business: s3://yelp-aai540-group6/yelp-dataset/json/yelp_academic_dataset_business.json -> s3://yelp-aai540-group6-athena-297628177412/athena-results-staging/yelp/json/business/yelp_academic_dataset_business.json
Staging review: s3://yelp-aai540-group6/yelp-dataset/json/yelp_academic_dataset_review.json -> s3://yelp-aai540-group6-athena-297628177412/athena-results-staging/yelp/json/review/yelp_academic_dataset_review.json
Staging user: s3://yelp-aai540-group6/yelp-dataset/json/yelp_academic_dataset_user.json -> s3://yelp-aai540-group6-athena-297628177412/athena-results-staging/yelp/json/user/yelp_academic_dataset_user.json
Staging checkin: s3://yelp-aai540-group6/yelp-dataset/json/yelp_academic_dataset_checkin.json -> s3://yelp-aai540-group6-athena-297628177412/athena-results-staging/yelp/json/checkin/yelp_academic_dataset_checkin.json
Staging tip: s3://yelp-aai540-group6/yelp-dataset/json/yelp_academic_dataset_tip.json -> s3://yelp-aai540-group6-athena-297628177412/athena-resu

# Create the Athena Tables

While creating the tables, to increase query efficiency, we will create a temporary table from the JSON data and then create the actual tables while utilizing parquet files.

JSON parsing within the tables is extremely slow, so this ensures we can query the athena tables in a reasonable amount of time. After creating the tables from the parquet data, we drop the temp tables to save space.

## 🏢 `yelp.business` Table

This table defines the **core business metadata** for the Yelp dataset.  
It is an **external Athena table** backed by JSON files stored in S3 and uses the
`JsonSerDe` to parse semi-structured data.

Each row represents **one Yelp business**.

### 🔑 Identifiers & Location

| Field | Type | Description |
|------|------|-------------|
| `business_id` | `string` | Unique Yelp identifier for the business (primary key for joins). |
| `address` | `string` | Street address of the business. |
| `city` | `string` | City where the business is located. |
| `state` | `string` | Two-letter state or province code. |
| `postal_code` | `string` | ZIP or postal code. |
| `latitude` | `double` | Latitude coordinate of the business location. |
| `longitude` | `double` | Longitude coordinate of the business location. |

---

### 🏷️ Business Metadata

| Field | Type | Description |
|------|------|-------------|
| `name` | `string` | Business name as listed on Yelp. |
| `categories` | `string` | Comma-separated list of Yelp business categories. |
| `is_open` | `int` | Indicates whether the business is currently open (`1`) or closed (`0`). |

---

### ⭐ Reviews & Ratings

| Field | Type | Description |
|------|------|-------------|
| `stars` | `double` | Average Yelp star rating (1.0–5.0). |
| `review_count` | `int` | Total number of Yelp reviews for the business. |

---

### 🧩 Semi-Structured Fields

These fields contain nested or semi-structured data and are parsed as maps.

| Field | Type | Description |
|------|------|-------------|
| `attributes` | `map<string,string>` | Key–value map of business attributes (e.g., WiFi, parking, ambience). Many values are string-encoded booleans or nested JSON strings. |
| `hours` | `map<string,string>` | Mapping of day → opening hours (e.g., `"Monday" → "11:00-22:00"`). |

> ⚠️ These fields are intentionally left unflattened here and are processed later into
> curated feature tables (e.g., `business_attributes`).

In [17]:
business_location = dest_locations["business"]

# parquet_prefix
business_curated_prefix = f"{parquet_prefix}business"
business_curated_location = f"s3://{ATHENA_BUCKET}/{business_curated_prefix}"

print("Creating temporary table")
run_athena_query(f"""
CREATE EXTERNAL TABLE IF NOT EXISTS {ATHENA_DB}.business_temp (
  business_id string,
  name string,
  address string,
  city string,
  state string,
  postal_code string,
  latitude double,
  longitude double,
  stars double,
  review_count int,
  is_open int,
  attributes map<string,string>,
  categories string,
  hours map<string,string>
)
ROW FORMAT SERDE 'org.openx.data.jsonserde.JsonSerDe'
WITH SERDEPROPERTIES ('ignore.malformed.json'='true')
LOCATION '{business_location}'
TBLPROPERTIES ('has_encrypted_data'='false');
""", database=ATHENA_DB)

print("Creating table with parquets")
run_athena_query(f"""
CREATE TABLE {ATHENA_DB}.business
WITH (
  format = 'PARQUET',
  external_location = '{business_curated_location}'
) AS
SELECT
  business_id,
  name,
  address,
  city,
  state,
  postal_code,
  latitude,
  longitude,
  stars,
  review_count,
  is_open,
  attributes,
  categories,
  hours

FROM {ATHENA_DB}.business_temp;
""", database=ATHENA_DB)

print("drop temp table")
run_athena_query(f"DROP TABLE IF EXISTS {ATHENA_DB}.business_temp;", database=ATHENA_DB)

print(f"✅ Created table {ATHENA_DB}.business")

Creating temporary table
Creating table with parquets
drop temp table
✅ Created table yelp.business


## 📝 `yelp.review` Table

This table contains **individual user reviews** for Yelp businesses.  
Each row represents **one written review** associated with a business and a user.

The table is defined as an **external Athena table** over JSON files stored in S3 and
uses the `JsonSerDe` to parse semi-structured data.

---

### 🔑 Identifiers & Relationships

| Field | Type | Description |
|------|------|-------------|
| `review_id` | `string` | Unique Yelp identifier for the review (primary key). |
| `user_id` | `string` | Yelp identifier of the user who authored the review. |
| `business_id` | `string` | Yelp identifier of the reviewed business. Used to join with `yelp.business`. |

---

### ⭐ Review Ratings & Feedback Signals

| Field | Type | Description |
|------|------|-------------|
| `stars` | `double` | Star rating given by the user (1.0–5.0). |
| `useful` | `int` | Number of times other users marked the review as useful. |
| `funny` | `int` | Number of times other users marked the review as funny. |
| `cool` | `int` | Number of times other users marked the review as cool. |

> These fields provide **engagement signals** that can be used as proxies for review impact or quality.

---

### 🗒️ Review Content & Time

| Field | Type | Description |
|------|------|-------------|
| `text` | `string` | Full free-text content of the review. |
| `date` | `string` | Date the review was posted (ISO-like format, e.g., `YYYY-MM-DD`). |

> 🧠 The `text` field is typically used for **NLP tasks** such as sentiment analysis,
> topic modeling, or embedding generation.


In [18]:
review_location = dest_locations["review"]
# parquet_prefix
review_curated_prefix = f"{parquet_prefix}review"
review_curated_location = f"s3://{ATHENA_BUCKET}/{review_curated_prefix}"

print("Creating temporary table")
run_athena_query(f"""
CREATE EXTERNAL TABLE IF NOT EXISTS {ATHENA_DB}.review_temp (
  review_id string,
  user_id string,
  business_id string,
  stars double,
  useful int,
  funny int,
  cool int,
  text string,
  date string
)
ROW FORMAT SERDE 'org.openx.data.jsonserde.JsonSerDe'
WITH SERDEPROPERTIES ('ignore.malformed.json'='true')
LOCATION '{review_location}'
TBLPROPERTIES ('has_encrypted_data'='false');
""", database=ATHENA_DB)

print("Creating table with parquets")
run_athena_query(f"""
CREATE TABLE {ATHENA_DB}.review
WITH (
  format = 'PARQUET',
  external_location = '{review_curated_location}',
  partitioned_by = ARRAY['year']
) AS
SELECT
  review_id,
  user_id,
  business_id,
  stars,
  useful,
  funny,
  cool,
  text,
  date,
  CAST(substr(date, 1, 4) AS integer) AS year
  
FROM {ATHENA_DB}.review_temp
WHERE date IS NOT NULL;
""", database=ATHENA_DB)

print("drop temp table")
run_athena_query(f"DROP TABLE IF EXISTS {ATHENA_DB}.review_temp;", database=ATHENA_DB)

print(f"✅ Created table {ATHENA_DB}.review")

Creating temporary table
Creating table with parquets
drop temp table
✅ Created table yelp.review


## 👤 `yelp.user` Table

This table contains **Yelp user profile information** and historical engagement metrics.  
Each row represents **one Yelp user** who has written reviews on the platform.

The table is defined as an **external Athena table** over JSON files stored in S3 and
uses the `JsonSerDe` to parse semi-structured data.

---

### 🔑 User Identity

| Field | Type | Description |
|------|------|-------------|
| `user_id` | `string` | Unique Yelp identifier for the user (primary key). |
| `name` | `string` | Display name of the Yelp user. |
| `yelping_since` | `string` | Date the user joined Yelp (format: `YYYY-MM`). |

---

### 📊 Review Activity & Reputation

These fields describe **how active and influential** a user is on Yelp.

| Field | Type | Description |
|------|------|-------------|
| `review_count` | `int` | Total number of reviews written by the user. |
| `average_stars` | `double` | Average star rating given by the user across all reviews. |
| `fans` | `int` | Number of other users who have marked this user as a fan. |


---

### 🤝 Social & Community Features

| Field | Type | Description |
|------|------|-------------|
| `friends` | `array<string>` | List of Yelp user IDs representing the user’s friends. |
| `elite` | `array<string>` | Years in which the user was part of Yelp’s “Elite Squad”. |

> 🧠 Elite users are often considered **high-quality reviewers** and may be weighted differently in analysis.

---

### 👍 Review Feedback Metrics

These fields count how other users have reacted to this user’s reviews.

| Field | Type | Description |
|------|------|-------------|
| `useful` | `int` | Total number of “useful” votes across all reviews. |
| `funny` | `int` | Total number of “funny” votes across all reviews. |
| `cool` | `int` | Total number of “cool” votes across all reviews. |

---

### 🌟 Compliment Metrics

Compliments are positive endorsements given by other users. These metrics help quantify **user reputation and content quality**.

| Field | Type | Description |
|------|------|-------------|
| `compliment_hot` | `int` | Compliments for “hot” reviews. |
| `compliment_more` | `int` | Compliments encouraging more reviews. |
| `compliment_profile` | `int` | Compliments on the user’s profile. |
| `compliment_cute` | `int` | Compliments marked as “cute”. |
| `compliment_list` | `int` | Compliments for curated lists. |
| `compliment_note` | `int` | Compliments for thoughtful notes. |
| `compliment_plain` | `int` | Compliments for plain but useful reviews. |
| `compliment_cool` | `int` | Compliments for “cool” content. |
| `compliment_funny` | `int` | Compliments for humorous content. |
| `compliment_writer` | `int` | Compliments for writing quality. |
| `compliment_photos` | `int` | Compliments for photos contributed by the user. |


In [19]:
user_location = dest_locations["user"]

# parquet_prefix
user_curated_prefix = f"{parquet_prefix}user"
user_curated_location = f"s3://{ATHENA_BUCKET}/{user_curated_prefix}"

print("Create temp table")
run_athena_query(f"""
CREATE EXTERNAL TABLE IF NOT EXISTS {ATHENA_DB}.user_temp (
  user_id string,
  name string,
  review_count int,
  yelping_since string,
  friends array<string>,
  useful int,
  funny int,
  cool int,
  fans int,
  elite array<string>,
  average_stars double,
  compliment_hot int,
  compliment_more int,
  compliment_profile int,
  compliment_cute int,
  compliment_list int,
  compliment_note int,
  compliment_plain int,
  compliment_cool int,
  compliment_funny int,
  compliment_writer int,
  compliment_photos int
)
ROW FORMAT SERDE 'org.openx.data.jsonserde.JsonSerDe'
WITH SERDEPROPERTIES ('ignore.malformed.json'='true')
LOCATION '{user_location}'
TBLPROPERTIES ('has_encrypted_data'='false');
""", database=ATHENA_DB)

print("Create Parquet table")
run_athena_query(f"""
CREATE TABLE {ATHENA_DB}.user
WITH (
  format = 'PARQUET',
  external_location = '{user_curated_location}'
) AS
SELECT
  user_id,
  name,
  review_count,
  yelping_since,
  friends,
  useful,
  funny,
  cool,
  fans,
  elite,
  average_stars,
  compliment_hot,
  compliment_more,
  compliment_profile,
  compliment_cute,
  compliment_list,
  compliment_note,
  compliment_plain,
  compliment_cool,
  compliment_funny,
  compliment_writer,
  compliment_photos
FROM {ATHENA_DB}.user_temp;
""", database=ATHENA_DB)

print("Drop temp table")
run_athena_query(f"DROP TABLE IF EXISTS {ATHENA_DB}.user_temp;", database=ATHENA_DB)

print(f"✅ Created table {ATHENA_DB}.user")

Create temp table
Create Parquet table
Drop temp table
✅ Created table yelp.user


## 📍 `yelp.checkin` Table

This table records **user check-in activity** for Yelp businesses.  
Each row represents **aggregated check-in timestamps** associated with a single business.

Unlike reviews, check-ins do **not** identify individual users; they capture **visit behavior over time**.

---

### 🔑 Identifiers

| Field | Type | Description |
|------|------|-------------|
| `business_id` | `string` | Yelp identifier for the business being checked into. Used to join with `yelp.business`. |

---

### ⏱️ Temporal Activity Data

| Field | Type | Description |
|------|------|-------------|
| `date` | `string` | Comma-separated list of check-in timestamps (e.g., `YYYY-MM-DD HH:MM:SS`). Each timestamp represents a recorded check-in event. |

> ⚠️ This field is **not a single timestamp**. It must be parsed or exploded to analyze hourly, daily, or weekly traffic patterns.


In [20]:
checkin_location = dest_locations["checkin"]

# parquet_prefix
checkin_curated_prefix = f"{parquet_prefix}checkin"
checkin_curated_location = f"s3://{ATHENA_BUCKET}/{checkin_curated_prefix}"

print("Create temp table")
run_athena_query(f"""
CREATE EXTERNAL TABLE IF NOT EXISTS {ATHENA_DB}.checkin_temp (
  business_id string,
  date string
)
ROW FORMAT SERDE 'org.openx.data.jsonserde.JsonSerDe'
WITH SERDEPROPERTIES ('ignore.malformed.json'='true')
LOCATION '{checkin_location}'
TBLPROPERTIES ('has_encrypted_data'='false');
""", database=ATHENA_DB)

print("Create Parquet table")
run_athena_query(f"""
CREATE TABLE {ATHENA_DB}.checkin
WITH (
  format = 'PARQUET',
  external_location = '{checkin_curated_location}'
) AS
SELECT
  business_id,
  date
FROM {ATHENA_DB}.checkin_temp;
""", database=ATHENA_DB)

print("Drop temp table")
run_athena_query(f"DROP TABLE IF EXISTS {ATHENA_DB}.checkin_temp;", database=ATHENA_DB)


print(f"✅ Created table {ATHENA_DB}.checkin")

Create temp table
Create Parquet table
Drop temp table
✅ Created table yelp.checkin


## 💬 `yelp.tip` Table

This table contains **short user-submitted tips** for Yelp businesses.  
Tips are lightweight comments or advice, often written without a full star rating.

Each row represents **one tip** authored by a user for a specific business.

---

### 🔑 Identifiers & Relationships

| Field | Type | Description |
|------|------|-------------|
| `user_id` | `string` | Yelp identifier of the user who wrote the tip. |
| `business_id` | `string` | Yelp identifier of the business the tip refers to. Used to join with `yelp.business`. |

---

### 🗒️ Tip Content & Engagement

| Field | Type | Description |
|------|------|-------------|
| `text` | `string` | Free-text content of the tip. Usually brief and advice-oriented. |
| `compliment_count` | `int` | Number of compliments received for the tip from other users. Serves as a proxy for usefulness or popularity. |

---

### ⏱️ Temporal Information

| Field | Type | Description |
|------|------|-------------|
| `date` | `string` | Date the tip was posted (typically in `YYYY-MM-DD` format). |


In [21]:
tip_location = dest_locations["tip"]

# parquet_prefix
tip_curated_prefix = f"{parquet_prefix}tip"
tip_curated_location = f"s3://{ATHENA_BUCKET}/{tip_curated_prefix}"

print("Create temp table")
run_athena_query(f"""
CREATE EXTERNAL TABLE IF NOT EXISTS {ATHENA_DB}.tip_temp (
  user_id string,
  business_id string,
  text string,
  date string,
  compliment_count int
)
ROW FORMAT SERDE 'org.openx.data.jsonserde.JsonSerDe'
WITH SERDEPROPERTIES ('ignore.malformed.json'='true')
LOCATION '{tip_location}'
TBLPROPERTIES ('has_encrypted_data'='false');
""", database=ATHENA_DB)

print("Create Parquet table")
run_athena_query(f"""
CREATE TABLE {ATHENA_DB}.tip
WITH (
  format = 'PARQUET',
  external_location = '{tip_curated_location}',
  partitioned_by = ARRAY['year']
) AS
SELECT
  user_id,
  business_id,
  text,
  date,
  compliment_count,
  CAST(substr(date, 1, 4) AS integer) AS year

FROM {ATHENA_DB}.tip_temp
WHERE date IS NOT NULL;
""", database=ATHENA_DB)

print("Drop temp table")
run_athena_query(f"DROP TABLE IF EXISTS {ATHENA_DB}.tip_temp;", database=ATHENA_DB)

print(f"✅ Created table {ATHENA_DB}.tip")

Create temp table
Create Parquet table
Drop temp table
✅ Created table yelp.tip


## 🧱 `yelp.business_attributes`

This cell builds the **`yelp.business_attributes` table**, a **fully normalized, analytics-ready feature table** derived from the raw `yelp.business` data.

It converts Yelp’s highly inconsistent, semi-structured `attributes` field into **typed, flattened columns** suitable for SQL analytics and machine learning.

The table is written as **Parquet** to S3 using a **CTAS (CREATE TABLE AS SELECT)** statement.

---

### 🎯 Purpose of This Table

The raw Yelp `attributes` and `hours` column:
- contains mixed data types
- uses stringified Python dictionaries
- includes inconsistent boolean values (`True`, `False`, `None`)
- embeds nested structures (parking, ambience, meals, etc.)

This table:
- **normalizes** all attributes
- **parses nested maps**
- **flattens features into columns**
- **preserves NULL vs FALSE semantics**
- produces a **single row per business** with clean features

This is the primary **feature dimension table** for downstream analysis and modeling.

---

### 🗂️ Output Characteristics

- **Format:** Parquet (columnar, efficient for Athena + ML)
- **Location:**  
  `s3://<ATHENA_BUCKET>/<parquet_prefix>/business`
- **Grain:** One row per `business_id`
- **Source:** `yelp.business`

Dropping this table does **not** delete the source business data.

---

### 🧩 Processing Pipeline Overview

#### 1️⃣ Normalization (`normalized` CTE)

- Cleans `attributes` keys and values by removing:
  - Python unicode wrappers (`u'...'`)
  - quoted string wrappers (`'...'`)
- Produces a clean `attrs` map:
- Retains `hours` for later feature extraction

This step ensures all downstream parsing operates on consistent input.

---

#### 2️⃣ Structured Parsing (`parsed` CTE)

Nested Yelp attributes stored as **stringified dictionaries** are converted into typed maps:

| Attribute Group | Output Type |
|-----------------|------------|
| BusinessParking | `map<string, boolean>` |
| Ambience | `map<string, boolean>` |
| GoodForMeal | `map<string, boolean>` |
| BestNights | `map<string, boolean>` |
| HairSpecializesIn | `map<string, boolean>` |
| DietaryRestrictions | `map<string, boolean>` |

Each map:
- normalizes `True / False / None`
- safely parses JSON using `TRY(...)`
- yields `NULL` instead of failing on malformed records

---

#### 3️⃣ Attribute Flattening (Final SELECT)

The final `SELECT` performs three key transformations:

##### 🟦 Scalar Attributes
Single-value attributes (e.g. `wifi`, `alcohol`, `noiselevel`) are:
- normalized so literal `"None"` becomes SQL `NULL`
- preserved as strings for categorical modeling

Example pattern:

```sql 
CASE WHEN value IS NULL OR lower(value) = 'none' THEN NULL ELSE value END
```

---

##### 🟩 Nested Boolean Feature Groups
Each parsed map is flattened into **explicit boolean columns**, e.g.:

- Parking: `parking_garage`, `parking_street`, …
- Ambience: `ambience_trendy`, `ambience_casual`, …
- Meals: `good_for_dinner`, `good_for_brunch`, …
- Best nights: `bestnight_friday`, `bestnight_saturday`, …
- Dietary flags: `vegan`, `gluten_free`, …
- Hair services: `hair_curly`, `hair_coloring`, …

Missing keys remain `NULL`, preserving signal quality.

---

##### 🟨 Hours & Temporal Features
Business hours are extracted from the `hours` map:

- Raw per-day opening hours:
  - `hours_monday`, `hours_tuesday`, …
- Derived features:
  - `open_days_count`
  - `open_on_weekend`

These features capture **operational availability**.


This table is a curated, analytics-ready **one-row-per-business** feature table derived from `yelp.business.attributes` plus `yelp.business.hours`.  
It includes (1) cleaned scalar attributes, (2) flattened nested attribute groups (parking/ambience/meals/etc.), and (3) hours-based operational features.

---

### 1) Core Identifier

| Field | Type | Description |
|------|------|-------------|
| `business_id` | `string` | Unique Yelp business identifier. Primary join key to `yelp.business`, `yelp.review`, `yelp.checkin`, and `yelp.tip`. |

---

### 2) Cleaned Scalar Attribute Columns (from `attributes` map)

> These are extracted from the `attributes` map and normalized so literal `"None"` values become SQL `NULL`.  
> Most remain **categorical strings** (often representing booleans like `"True"/"False"`, or enums like `"free"/"paid"`).

| Field | Type | Description |
|------|------|-------------|
| `acceptsinsurance` | `string` | Whether the business accepts insurance (commonly `"True"`/`"False"`). |
| `agesallowed` | `string` | Age restrictions/allowed audience (values vary by business/category). |
| `alcohol` | `string` | Alcohol service type (e.g., `"none"`, `"beer_and_wine"`, `"full_bar"`). |
| `bikeparking` | `string` | Whether bike parking is available (often `"True"`/`"False"`). |
| `businessacceptsbitcoin` | `string` | Whether the business accepts Bitcoin (often `"True"`/`"False"`). |
| `businessacceptscreditcards` | `string` | Whether credit cards are accepted (often `"True"`/`"False"`). |
| `byappointmentonly` | `string` | Whether the business is appointment-only (often `"True"`/`"False"`). |
| `byob` | `string` | Whether customers can bring their own alcohol (often `"True"`/`"False"`). |
| `byobcorkage` | `string` | Corkage policy for BYOB (varies; can be boolean-like or descriptive). |
| `caters` | `string` | Whether catering is offered (often `"True"`/`"False"`). |
| `coatcheck` | `string` | Whether coat check is available (often `"True"`/`"False"`). |
| `corkage` | `string` | Corkage policy indicator/details (often boolean-like or descriptive). |
| `dogsallowed` | `string` | Whether dogs are allowed (often `"True"`/`"False"`). |
| `drivethru` | `string` | Whether a drive-thru is available (often `"True"`/`"False"`). |
| `goodfordancing` | `string` | Whether the business is good for dancing (often `"True"`/`"False"`). |
| `goodforkids` | `string` | Whether the business is good for kids (often `"True"`/`"False"`). |
| `happyhour` | `string` | Whether happy hour is offered (often `"True"`/`"False"`). |
| `hastv` | `string` | Whether the business has TVs (often `"True"`/`"False"`). |
| `music` | `string` | Music-related attribute (often a nested/encoded structure; may be a stringified map). |
| `noiselevel` | `string` | Noise level category (e.g., `"quiet"`, `"average"`, `"loud"`, `"very_loud"`). |
| `open24hours` | `string` | Whether the business is open 24 hours (often `"True"`/`"False"`). |
| `outdoorseating` | `string` | Whether outdoor seating is available (often `"True"`/`"False"`). |
| `restaurantsattire` | `string` | Attire expectation (e.g., `"casual"`, `"dressy"`). |
| `restaurantscounterservice` | `string` | Whether counter service is provided (often `"True"`/`"False"`). |
| `restaurantsdelivery` | `string` | Whether delivery is offered (often `"True"`/`"False"`). |
| `restaurantsgoodforgroups` | `string` | Whether the business is good for groups (often `"True"`/`"False"`). |
| `restaurantspricerange2` | `string` | Price range indicator (often `"1"`–`"4"` as strings). |
| `restaurantsreservations` | `string` | Whether reservations are accepted (often `"True"`/`"False"`). |
| `restaurantstableservice` | `string` | Whether table service is provided (often `"True"`/`"False"`). |
| `restaurantstakeout` | `string` | Whether takeout is offered (often `"True"`/`"False"`). |
| `smoking` | `string` | Smoking policy category (varies; may be `"no"`, `"yes"`, or descriptive). |
| `wheelchairaccessible` | `string` | Whether wheelchair access is available (often `"True"`/`"False"`). |
| `wifi` | `string` | WiFi availability/category (commonly `"free"`, `"paid"`, `"no"`). |

---

### 3) Parking Features (from `attributes['businessparking']`)

> Parsed from the nested `BusinessParking` dict into booleans.  
> Values are `TRUE`/`FALSE` when specified, otherwise `NULL`.

| Field | Type | Description |
|------|------|-------------|
| `parking_garage` | `boolean` | Garage parking available. |
| `parking_street` | `boolean` | Street parking available. |
| `parking_validated` | `boolean` | Validated parking available. |
| `parking_lot` | `boolean` | Parking lot available. |
| `parking_valet` | `boolean` | Valet parking available. |

---

### 4) Ambience Features (from `attributes['ambience']`)

> Parsed from the nested ambience dict into booleans.

| Field | Type | Description |
|------|------|-------------|
| `ambience_divey` | `boolean` | Divey atmosphere. |
| `ambience_hipster` | `boolean` | Hipster vibe. |
| `ambience_casual` | `boolean` | Casual vibe. |
| `ambience_touristy` | `boolean` | Touristy atmosphere. |
| `ambience_trendy` | `boolean` | Trendy vibe. |
| `ambience_intimate` | `boolean` | Intimate atmosphere. |
| `ambience_romantic` | `boolean` | Romantic atmosphere. |
| `ambience_classy` | `boolean` | Classy atmosphere. |
| `ambience_upscale` | `boolean` | Upscale atmosphere. |

---

### 5) GoodForMeal Features (from `attributes['goodformeal']`)

> Parsed from the nested meal suitability dict into booleans.

| Field | Type | Description |
|------|------|-------------|
| `good_for_dessert` | `boolean` | Suitable for dessert. |
| `good_for_latenight` | `boolean` | Suitable for late night dining. |
| `good_for_lunch` | `boolean` | Suitable for lunch. |
| `good_for_dinner` | `boolean` | Suitable for dinner. |
| `good_for_brunch` | `boolean` | Suitable for brunch. |
| `good_for_breakfast` | `boolean` | Suitable for breakfast. |

---

### 6) BestNights Features (from `attributes['bestnights']`)

> Parsed from the nested “best nights to go” dict into booleans.

| Field | Type | Description |
|------|------|-------------|
| `bestnight_monday` | `boolean` | Monday is considered a best night. |
| `bestnight_tuesday` | `boolean` | Tuesday is considered a best night. |
| `bestnight_wednesday` | `boolean` | Wednesday is considered a best night. |
| `bestnight_thursday` | `boolean` | Thursday is considered a best night. |
| `bestnight_friday` | `boolean` | Friday is considered a best night. |
| `bestnight_saturday` | `boolean` | Saturday is considered a best night. |
| `bestnight_sunday` | `boolean` | Sunday is considered a best night. |

---

### 7) HairSpecializesIn Features (from `attributes['hairspecializesin']`)

> Parsed from the nested hair services specialization dict into booleans.

| Field | Type | Description |
|------|------|-------------|
| `hair_africanamerican` | `boolean` | Specializes in African-American hair services. |
| `hair_asian` | `boolean` | Specializes in Asian hair services. |
| `hair_coloring` | `boolean` | Specializes in hair coloring. |
| `hair_curly` | `boolean` | Specializes in curly hair services. |
| `hair_extensions` | `boolean` | Offers/specializes in hair extensions. |
| `hair_kids` | `boolean` | Offers/specializes in kids hair services. |
| `hair_perms` | `boolean` | Offers/specializes in perms. |
| `hair_straightperms` | `boolean` | Offers/specializes in straight perms. |

---

### 8) Dietary Restriction Features (from `attributes['dietaryrestrictions']`)

> Parsed from the nested dietary restriction dict into booleans.

| Field | Type | Description |
|------|------|-------------|
| `dairy_free` | `boolean` | Dairy-free options available. |
| `gluten_free` | `boolean` | Gluten-free options available. |
| `vegan` | `boolean` | Vegan options available. |
| `kosher` | `boolean` | Kosher options available. |
| `halal` | `boolean` | Halal options available. |
| `soy_free` | `boolean` | Soy-free options available. |
| `vegetarian` | `boolean` | Vegetarian options available. |

---

### 9) Hours Features (from `business.hours`)

> Hours are extracted from the `hours` map in `yelp.business`.  
> Values are typically formatted like `"HH:MM-HH:MM"`. Missing days are `NULL`.

| Field | Type | Description |
|------|------|-------------|
| `hours_monday` | `string` | Monday operating hours (e.g., `"11:00-22:00"`). |
| `hours_tuesday` | `string` | Tuesday operating hours. |
| `hours_wednesday` | `string` | Wednesday operating hours. |
| `hours_thursday` | `string` | Thursday operating hours. |
| `hours_friday` | `string` | Friday operating hours. |
| `hours_saturday` | `string` | Saturday operating hours. |
| `hours_sunday` | `string` | Sunday operating hours. |
| `open_days_count` | `integer` | Count of days with defined hours (`CARDINALITY(map_keys(hours))`). |
| `open_on_weekend` | `boolean` | `TRUE` if Saturday or Sunday hours exist; otherwise `FALSE`. |

---

#### ⚠️ Notes on NULL Semantics

- For nested maps (parking/ambience/meals/etc.), `NULL` means:
  - the key was missing, or
  - the original value was `None`
- `FALSE` means Yelp explicitly indicated the attribute is not true.
- For scalar attributes, literal `"None"` strings are normalized to SQL `NULL`.


In [22]:
business_attributes_location = f"s3://{ATHENA_BUCKET}/{parquet_prefix}business_attributes"

run_athena_query(f"""
CREATE TABLE {ATHENA_DB}.business_attributes
WITH (
  format = 'PARQUET',
  external_location = '{business_attributes_location}'
) AS
WITH normalized AS (
  SELECT
    business_id,
    hours,

    -- Normalize u'...' and '...' wrappers on keys + values
    map_from_entries(
      transform(
        map_entries(attributes),
        e -> CAST(
          ROW(
            regexp_replace(CAST(e[1] AS varchar), '^u?''(.*)''$', '$1'),
            regexp_replace(CAST(e[2] AS varchar), '^u?''(.*)''$', '$1')
          ) AS ROW(varchar, varchar)
        )
      )
    ) AS attrs
  FROM {ATHENA_DB}.business
  WHERE attributes IS NOT NULL
),
parsed AS (
  SELECT
    business_id,
    hours,
    
    -- Convert literal "None" (any case) to NULL for all attribute lookups via helper expression pattern below
    attrs,

    -- Parse BusinessParking
    TRY(
      CAST(
        json_parse(
          replace(
            replace(
              replace(
                replace(
                  regexp_replace(attrs['businessparking'], 'u''(.*?)''', '"$1"'),
                  '''', '"'
                ),
                'False', 'false'
              ),
              'True', 'true'
            ),
            'None', 'null'
          )
        ) AS map(varchar, boolean)
      )
    ) AS parking_map,

    -- Parse Ambience 
    TRY(
      CAST(
        json_parse(
          replace(
            replace(
              replace(
                replace(
                  -- normalize u'...' keys inside the string
                  regexp_replace(attrs['ambience'], 'u''(.*?)''', '"$1"'),
                  '''', '"'
                ),
                'False', 'false'
              ),
              'True', 'true'
            ),
            'None', 'null'
          )
        ) AS map(varchar, boolean)
      )
    ) AS ambience_map,

    -- Parse GoodForMeal
    TRY(
      CAST(
        json_parse(
          replace(
            replace(
              replace(
                replace(
                  -- normalize u'...' keys
                  regexp_replace(attrs['goodformeal'], 'u''(.*?)''', '"$1"'),
                  '''', '"'
                ),
                'False', 'false'
              ),
              'True', 'true'
            ),
            'None', 'null'
          )
        ) AS map(varchar, boolean)
      )
    ) AS goodformeal_map,

    -- Parse BestNights
    TRY(
      CAST(
        json_parse(
          replace(
            replace(
              replace(
                replace(
                  -- normalize u'...' keys
                  regexp_replace(attrs['bestnights'], 'u''(.*?)''', '"$1"'),
                  '''', '"'
                ),
                'False', 'false'
              ),
              'True', 'true'
            ),
            'None', 'null'
          )
        ) AS map(varchar, boolean)
      )
    ) AS bestnights_map,

    -- Parse HairSpecializesIn
    TRY(
      CAST(
        json_parse(
          replace(
            replace(
              replace(
                replace(
                  -- normalize u'...' keys
                  regexp_replace(attrs['hairspecializesin'], 'u''(.*?)''', '"$1"'),
                  '''', '"'
                ),
                'False', 'false'
              ),
              'True', 'true'
            ),
            'None', 'null'
          )
        ) AS map(varchar, boolean)
      )
    ) AS hairspecializesin_map,
    
    -- Parse DietaryRestrictions
    TRY(
      CAST(
        json_parse(
          replace(
            replace(
              replace(
                replace(
                  -- normalize u'...' keys
                  regexp_replace(attrs['dietaryrestrictions'], 'u''(.*?)''', '"$1"'),
                  '''', '"'
                ),
                'False', 'false'
              ),
              'True', 'true'
            ),
            'None', 'null'
          )
        ) AS map(varchar, boolean)
      )
    ) AS dietaryrestrictions_map
  FROM normalized
)
SELECT
    business_id,
    
    -- Helper pattern: NULLIF(lower(x),'none') but preserving original case when not none
    CASE WHEN attrs['acceptsinsurance'] IS NULL OR lower(attrs['acceptsinsurance']) = 'none' THEN NULL ELSE attrs['acceptsinsurance'] END AS acceptsinsurance,
    CASE WHEN attrs['agesallowed'] IS NULL OR lower(attrs['agesallowed']) = 'none' THEN NULL ELSE attrs['agesallowed'] END AS agesallowed,
    CASE WHEN attrs['alcohol'] IS NULL OR lower(attrs['alcohol']) = 'none' THEN NULL ELSE attrs['alcohol'] END AS alcohol,
    CASE WHEN attrs['bikeparking'] IS NULL OR lower(attrs['bikeparking']) = 'none' THEN NULL ELSE attrs['bikeparking'] END AS bikeparking,
    CASE WHEN attrs['businessacceptsbitcoin'] IS NULL OR lower(attrs['businessacceptsbitcoin']) = 'none' THEN NULL ELSE attrs['businessacceptsbitcoin'] END AS businessacceptsbitcoin,
    CASE WHEN attrs['businessacceptscreditcards'] IS NULL OR lower(attrs['businessacceptscreditcards']) = 'none' THEN NULL ELSE attrs['businessacceptscreditcards'] END AS businessacceptscreditcards,
    CASE WHEN attrs['byappointmentonly'] IS NULL OR lower(attrs['byappointmentonly']) = 'none' THEN NULL ELSE attrs['byappointmentonly'] END AS byappointmentonly,
    CASE WHEN attrs['byob'] IS NULL OR lower(attrs['byob']) = 'none' THEN NULL ELSE attrs['byob'] END AS byob,
    CASE WHEN attrs['byobcorkage'] IS NULL OR lower(attrs['byobcorkage']) = 'none' THEN NULL ELSE attrs['byobcorkage'] END AS byobcorkage,
    CASE WHEN attrs['caters'] IS NULL OR lower(attrs['caters']) = 'none' THEN NULL ELSE attrs['caters'] END AS caters,
    CASE WHEN attrs['coatcheck'] IS NULL OR lower(attrs['coatcheck']) = 'none' THEN NULL ELSE attrs['coatcheck'] END AS coatcheck,
    CASE WHEN attrs['corkage'] IS NULL OR lower(attrs['corkage']) = 'none' THEN NULL ELSE attrs['corkage'] END AS corkage,
    CASE WHEN attrs['dogsallowed'] IS NULL OR lower(attrs['dogsallowed']) = 'none' THEN NULL ELSE attrs['dogsallowed'] END AS dogsallowed,
    CASE WHEN attrs['drivethru'] IS NULL OR lower(attrs['drivethru']) = 'none' THEN NULL ELSE attrs['drivethru'] END AS drivethru,
    CASE WHEN attrs['goodfordancing'] IS NULL OR lower(attrs['goodfordancing']) = 'none' THEN NULL ELSE attrs['goodfordancing'] END AS goodfordancing,
    CASE WHEN attrs['goodforkids'] IS NULL OR lower(attrs['goodforkids']) = 'none' THEN NULL ELSE attrs['goodforkids'] END AS goodforkids,
    CASE WHEN attrs['happyhour'] IS NULL OR lower(attrs['happyhour']) = 'none' THEN NULL ELSE attrs['happyhour'] END AS happyhour,
    CASE WHEN attrs['hastv'] IS NULL OR lower(attrs['hastv']) = 'none' THEN NULL ELSE attrs['hastv'] END AS hastv,
    CASE WHEN attrs['music'] IS NULL OR lower(attrs['music']) = 'none' THEN NULL ELSE attrs['music'] END AS music,
    CASE WHEN attrs['noiselevel'] IS NULL OR lower(attrs['noiselevel']) = 'none' THEN NULL ELSE attrs['noiselevel'] END AS noiselevel,
    CASE WHEN attrs['open24hours'] IS NULL OR lower(attrs['open24hours']) = 'none' THEN NULL ELSE attrs['open24hours'] END AS open24hours,
    CASE WHEN attrs['outdoorseating'] IS NULL OR lower(attrs['outdoorseating']) = 'none' THEN NULL ELSE attrs['outdoorseating'] END AS outdoorseating,
    CASE WHEN attrs['restaurantsattire'] IS NULL OR lower(attrs['restaurantsattire']) = 'none' THEN NULL ELSE attrs['restaurantsattire'] END AS restaurantsattire,
    CASE WHEN attrs['restaurantscounterservice'] IS NULL OR lower(attrs['restaurantscounterservice']) = 'none' THEN NULL ELSE attrs['restaurantscounterservice'] END AS restaurantscounterservice,
    CASE WHEN attrs['restaurantsdelivery'] IS NULL OR lower(attrs['restaurantsdelivery']) = 'none' THEN NULL ELSE attrs['restaurantsdelivery'] END AS restaurantsdelivery,
    CASE WHEN attrs['restaurantsgoodforgroups'] IS NULL OR lower(attrs['restaurantsgoodforgroups']) = 'none' THEN NULL ELSE attrs['restaurantsgoodforgroups'] END AS restaurantsgoodforgroups,
    CASE WHEN attrs['restaurantspricerange2'] IS NULL OR lower(attrs['restaurantspricerange2']) = 'none' THEN NULL ELSE attrs['restaurantspricerange2'] END AS restaurantspricerange2,
    CASE WHEN attrs['restaurantsreservations'] IS NULL OR lower(attrs['restaurantsreservations']) = 'none' THEN NULL ELSE attrs['restaurantsreservations'] END AS restaurantsreservations,
    CASE WHEN attrs['restaurantstableservice'] IS NULL OR lower(attrs['restaurantstableservice']) = 'none' THEN NULL ELSE attrs['restaurantstableservice'] END AS restaurantstableservice,
    CASE WHEN attrs['restaurantstakeout'] IS NULL OR lower(attrs['restaurantstakeout']) = 'none' THEN NULL ELSE attrs['restaurantstakeout'] END AS restaurantstakeout,
    CASE WHEN attrs['smoking'] IS NULL OR lower(attrs['smoking']) = 'none' THEN NULL ELSE attrs['smoking'] END AS smoking,
    CASE WHEN attrs['wheelchairaccessible'] IS NULL OR lower(attrs['wheelchairaccessible']) = 'none' THEN NULL ELSE attrs['wheelchairaccessible'] END AS wheelchairaccessible,
    CASE WHEN attrs['wifi'] IS NULL OR lower(attrs['wifi']) = 'none' THEN NULL ELSE attrs['wifi'] END AS wifi,
    
    -- Parking
    parking_map['garage']    AS parking_garage,
    parking_map['street']    AS parking_street,
    parking_map['validated'] AS parking_validated,
    parking_map['lot']       AS parking_lot,
    parking_map['valet']     AS parking_valet,
    
    -- Ambience
    ambience_map['divey']     AS ambience_divey,
    ambience_map['hipster']  AS ambience_hipster,
    ambience_map['casual']   AS ambience_casual,
    ambience_map['touristy'] AS ambience_touristy,
    ambience_map['trendy']   AS ambience_trendy,
    ambience_map['intimate'] AS ambience_intimate,
    ambience_map['romantic'] AS ambience_romantic,
    ambience_map['classy']   AS ambience_classy,
    ambience_map['upscale']  AS ambience_upscale,

    -- GoodForMeal
    goodformeal_map['dessert']    AS good_for_dessert,
    goodformeal_map['latenight'] AS good_for_latenight,
    goodformeal_map['lunch']     AS good_for_lunch,
    goodformeal_map['dinner']    AS good_for_dinner,
    goodformeal_map['brunch']    AS good_for_brunch,
    goodformeal_map['breakfast'] AS good_for_breakfast,

    -- BestNights
    bestnights_map['monday']    AS bestnight_monday,
    bestnights_map['tuesday']   AS bestnight_tuesday,
    bestnights_map['wednesday'] AS bestnight_wednesday,
    bestnights_map['thursday']  AS bestnight_thursday,
    bestnights_map['friday']    AS bestnight_friday,
    bestnights_map['saturday']  AS bestnight_saturday,
    bestnights_map['sunday']    AS bestnight_sunday,

    -- HairSpecializesIn
    hairspecializesin_map['africanamerican'] AS hair_africanamerican,
    hairspecializesin_map['asian']           AS hair_asian,
    hairspecializesin_map['coloring']        AS hair_coloring,
    hairspecializesin_map['curly']           AS hair_curly,
    hairspecializesin_map['extensions']      AS hair_extensions,
    hairspecializesin_map['kids']            AS hair_kids,
    hairspecializesin_map['perms']           AS hair_perms,
    hairspecializesin_map['straightperms']   AS hair_straightperms,

    -- DietaryRestrictions
    dietaryrestrictions_map['dairy-free']      AS dairy_free,
    dietaryrestrictions_map['gluten-free']    AS gluten_free,
    dietaryrestrictions_map['vegan']           AS vegan,
    dietaryrestrictions_map['kosher']          AS kosher,
    dietaryrestrictions_map['halal']           AS halal,
    dietaryrestrictions_map['soy-free']        AS soy_free,
    dietaryrestrictions_map['vegetarian']      AS vegetarian,

    -- Hours
    hours['monday']    AS hours_monday,
    hours['tuesday']   AS hours_tuesday,
    hours['wednesday'] AS hours_wednesday,
    hours['thursday']  AS hours_thursday,
    hours['friday']    AS hours_friday,
    hours['saturday']  AS hours_saturday,
    hours['sunday']    AS hours_sunday,

    CARDINALITY(map_keys(hours)) AS open_days_count,
    CASE WHEN hours['saturday'] IS NOT NULL OR hours['sunday'] IS NOT NULL THEN true ELSE false END AS open_on_weekend

FROM parsed;
""", database=ATHENA_DB)

print(f"✅ Built {ATHENA_DB}.business_attributes")
print("📍 Location:", business_attributes_location)

✅ Built yelp.business_attributes
📍 Location: s3://yelp-aai540-group6-athena-297628177412/parquet/business_attributes


# 🧹 Database & Storage Reset — Table and Data Cleanup

This cell performs a **full reset of the Yelp Athena environment**, removing both:
1. **Athena table definitions** from the Glue Data Catalog
2. **Materialized data files** in S3 associated with the curated `business_attributes` table

This ensures a **clean slate** before rebuilding tables or rerunning CTAS jobs.

**UNCOMMENT TO USE**

In [16]:
# TABLES = ["business", "review", "user", "checkin", "tip", "business_attributes"]
#
# for table in TABLES:
#     print(f"Dropping table: {ATHENA_DB}.{table}")
#     run_athena_query(
#         f"DROP TABLE IF EXISTS {ATHENA_DB}.{table};",
#         database="yelp"
#     )
#
# paginator = s3_private.get_paginator("list_objects_v2")
# to_delete = []
#
# for page in paginator.paginate(Bucket=ATHENA_BUCKET, Prefix=parquet_prefix):
#     for obj in page.get("Contents", []):
#         to_delete.append({"Key": obj["Key"]})
#
# if not to_delete:
#     print("✅ Nothing to delete under", f"s3://{ATHENA_BUCKET}/{parquet_prefix}")
# else:
#     # delete in batches of 1000 (S3 limit)
#     for i in range(0, len(to_delete), 1000):
#         s3_private.delete_objects(
#             Bucket=ATHENA_BUCKET,
#             Delete={"Objects": to_delete[i:i+1000]}
#         )
#    print(f"✅ Deleted {len(to_delete)} objects under s3://{ATHENA_BUCKET}/{parquet_prefix}")
#
# print("✅ All tables dropped.")

Dropping table: yelp.business
Dropping table: yelp.review
Dropping table: yelp.user
Dropping table: yelp.checkin
Dropping table: yelp.tip
Dropping table: yelp.business_attributes
✅ Nothing to delete under s3://yelp-aai540-group6-athena-297628177412/parquet/
✅ All tables dropped.


# Visualize table data

In [23]:
run_athena_query(f"SELECT * FROM {ATHENA_DB}.business LIMIT 2;", database=ATHENA_DB)

,business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,is_open,attributes,categories,hours
0,nHUJCK-Ek_lAVTSyEz9GEQ,Adventure Coast Fun Park,9227 County Line Rd,Spring Hill,FL,34608,28.4352625,-82.5669664,4.5,10,1,"{businessacceptscreditcards=True, goodforkids=...","American (New), Restaurants, Event Planning & ...","{sunday=11:0-21:0, saturday=11:0-21:0, tuesday..."
1,xHspDrdyq1g27yRDezB5yA,Kaiserman JCC,45 Haverford Rd,Philadelphia,PA,19096,39.982542,-75.268542,3.5,11,1,"{businessacceptscreditcards=True, byappointmen...","Preschools, Recreation Centers, Education, Gym...","{sunday=8:0-17:0, saturday=8:0-17:0, tuesday=5..."


In [24]:
run_athena_query(f"SELECT * FROM {ATHENA_DB}.checkin LIMIT 2;", database=ATHENA_DB)

,business_id,date
0,gVxh7PO5mDP5hAXG-XbtYA,"2021-11-20 04:25:59, 2021-12-04 23:51:53, 2022..."
1,gVzi6ytZ4Is8II27Irs26w,"2021-05-14 21:31:25, 2021-08-02 17:14:19, 2021..."


In [25]:
run_athena_query(f"SELECT * FROM {ATHENA_DB}.review LIMIT 2;", database=ATHENA_DB)

,review_id,user_id,business_id,stars,useful,funny,cool,text,date,year
0,i_fJQ_-9S9qhH2hl5bMIiw,FBRjdSizGuMyxQuSSwzarg,htO_nlxkEsYHzDrtbiUxew,3.0,3,0,0,"i was looking forward to a new coffee spot, bu...",2006-05-02 22:18:26,2006
1,zMgzXo2IFyajRqs60ppOsg,3xX-Et9I4Bi4wdRPWJkAFg,VGYWVWwuuNRzI2UcSNIw2g,5.0,1,0,0,"Yes, they actually make their own wine here in...",2006-01-01 20:01:40,2006


In [26]:
run_athena_query(f"SELECT * FROM {ATHENA_DB}.tip LIMIT 2;", database=ATHENA_DB)

,user_id,business_id,text,date,compliment_count,year
0,5S1chXV6YXXcvBXF1oRoNA,dPpHumLoB1GJQh0vpN3tNg,They checked out a used car I was looking at f...,2009-08-10 12:16:27,0,2009
1,A0Tkq4VV8dD6t8NbBxvBPA,9ggTidLF9LPNdyWdCGtrYg,The grill is only open until 7pm,2009-06-01 23:10:26,0,2009


In [27]:
run_athena_query(f"SELECT * FROM {ATHENA_DB}.user LIMIT 2;", database=ATHENA_DB)

,user_id,name,review_count,yelping_since,friends,useful,funny,cool,fans,elite,...,compliment_more,compliment_profile,compliment_cute,compliment_list,compliment_note,compliment_plain,compliment_cool,compliment_funny,compliment_writer,compliment_photos
0,W0V-VFWATCaxt8oGhZ-grQ,Samantha,34,2015-07-15 02:35:44,"[MIvDyZNghuRFqOfu2-oKow, SfKVbPxfLIASU9e3cVNLX...",34,5,10,1,None,...,1,0,0,0,0,0,2,2,1,0
1,H3TGQClC5t3yCykztuzyCw,Tiffany,3,2015-08-30 21:52:35,"[8IZ_yxuiaAJ9YqZE3SWHaQ, Nw4hE4K6NgpjL6yPsKLKq...",0,0,0,0,None,...,0,0,0,0,0,0,0,0,0,0


In [28]:
run_athena_query(f"SELECT * FROM {ATHENA_DB}.business_attributes LIMIT 2;", database=ATHENA_DB)

,business_id,acceptsinsurance,agesallowed,alcohol,bikeparking,businessacceptsbitcoin,businessacceptscreditcards,byappointmentonly,byob,byobcorkage,...,vegetarian,hours_monday,hours_tuesday,hours_wednesday,hours_thursday,hours_friday,hours_saturday,hours_sunday,open_days_count,open_on_weekend
0,nHUJCK-Ek_lAVTSyEz9GEQ,None,None,None,False,None,True,False,None,None,...,None,12:0-19:0,12:0-19:0,12:0-19:0,12:0-19:0,12:0-21:0,11:0-21:0,11:0-21:0,7,true
1,xHspDrdyq1g27yRDezB5yA,None,None,None,None,None,True,False,None,None,...,None,5:30-21:0,5:30-21:0,5:30-21:0,5:30-21:0,5:30-18:0,8:0-17:0,8:0-17:0,7,true
